In [1]:
from promptflow import PFClient

pf_client = PFClient()

from dotenv import load_dotenv

load_dotenv()

runtime = "automatic"
# load flow
flow = "../contoso-chat_hf" #"../contoso-chat"


eval_flow = "multi_flow/"   #"multi_flow_hf/"
data = "../data/salestestdata.jsonl"


In [2]:
# Add a question to test the base prompt flow.
question = "Can you tell me about your jackets?"
customerId = "4"
output_hf = pf_client.test(
    flow=flow, # Path to the flow directory
    inputs={ # Inputs to the flow
        "chat_history": [],
        "question": question,
        "customerId": customerId,
    },
)

output_hf["answer"] = "".join(list(output_hf["answer"]))

[2024-06-10 00:56:55 +0200][promptflow.core._connection][WARNING] - Please use connection.secrets[key] to access secrets.


Prompt flow service has started...


2024-06-10 00:56:55 +0200   26317 execution.flow     INFO     Start executing nodes in thread pool mode.


2024-06-10 00:56:55 +0200   26317 execution.flow     INFO     Start to run 5 nodes with concurrency level 16.


2024-06-10 00:56:55 +0200   26317 execution.flow     INFO     Executing node question_embedding. node run id: 6c939d43-f7f5-481f-b0d1-859a98a5490a_question_embedding_0


...

2024-06-10 00:56:56 +0200   26317 execution.flow     INFO     Node retrieve_documentation completes.


2024-06-10 00:56:56 +0200   26317 execution.flow     INFO     Executing node customer_prompt. node run id: 6c939d43-f7f5-481f-b0d1-859a98a5490a_customer_prompt_0


2024-06-10 00:56:56 +0200   26317 execution.flow     INFO     Node customer_prompt completes.


2024-06-10 00:56:56 +0200   26317 execution.flow     INFO     Executing node llm_response. node run id: 6c939d43-f7f5-481f-b0d1-859a98a5490a_llm_response_0


2024-06-10 00:56:57 +0200   26317 execution.flow     INFO     Node llm_response completes.


In [3]:
output_hf

{'answer': "Hi there 👋 SaraLee 🙏,\n\nI'd love 💕 sharing our awesome collection 😎of outerwear pieces tailored perfectly 🔩for adventurers like yourself 😉! \n\nConsidering previous buys such ⭐️as those fantastic `Cozynight`🛍️sleeping bags & comfyTrailblazerHikesPants💪, I think two fabulous choices would complement each item beautifully:\n\n1⃣ Our popularSummitBreezesJacket❄(Catalog ID :",
 'context': [{'id': '3',
   'title': 'Summit Breeze Jacket',
   'content': "Discover the joy of hiking with MountainStyle's Summit Breeze Jacket. This lightweight jacket is your perfect companion for outdoor adventures. Sporting a trail-ready, windproof design and a water-resistant fabric, it's ready to withstand any weather. The breathable polyester material and adjustable cuffs keep you comfortable, whether you're ascending a mountain or strolling through a park. And its sleek black color adds style to function. The jacket features a full-zip front closure, adjustable hood, and secure zippered pockets.

Test the groundedness of the prompt flow with the answer from the above question.

# Local Evaluation - Multiple Metrics 

Now use the same prompt flow and test it against the Multi Evaluation flow for groundedness, coherence, fluency, and relevance.

In [4]:
print("question",question)
print("context",output_hf["context"])
print("answer", output_hf["answer"])
test_multi_hf = pf_client.test(
    eval_flow,
    inputs={
        "question": question,
        "context": str(output_hf["context"]),
        "answer": output_hf["answer"],
    },
)

question Can you tell me about your jackets?
context [{'id': '3', 'title': 'Summit Breeze Jacket', 'content': "Discover the joy of hiking with MountainStyle's Summit Breeze Jacket. This lightweight jacket is your perfect companion for outdoor adventures. Sporting a trail-ready, windproof design and a water-resistant fabric, it's ready to withstand any weather. The breathable polyester material and adjustable cuffs keep you comfortable, whether you're ascending a mountain or strolling through a park. And its sleek black color adds style to function. The jacket features a full-zip front closure, adjustable hood, and secure zippered pockets. Experience the comfort of its inner lining and the convenience of its packable design. Crafted for night trekkers too, the jacket has reflective accents for enhanced visibility. Rugged yet chic, the Summit Breeze Jacket is more than a hiking essential, it's the gear that inspires you to reach new heights. Choose adventure, choose the Summit Breeze Jac

Prompt flow service has started...
2024-06-10 00:56:57 +0200   26317 execution.flow     INFO     Start executing nodes in thread pool mode.


2024-06-10 00:56:57 +0200   26317 execution.flow     INFO     Start to run 8 nodes with concurrency level 16.


2024-06-10 00:56:57 +0200   26317 execution.flow     INFO     Executing node coherence_score. node run id: 20d4dd69-1c8d-4663-8d00-1a05cc5dec81_coherence_score_0


2024-06-10 00:56:57 +0200   26317 execution.flow     INFO     Executing node fluency_score. node run id: 20d4dd69-1c8d-4663-8d00-1a05cc5dec81_fluency_score_0


...

2024-06-10 00:56:58 +0200   26317 execution.flow     INFO     Node groundedness_aggregate_variants_results completes.


2024-06-10 00:56:58 +0200   26317 execution.flow     INFO     [relevance_aggregate_variants_results in line None (index starts from 0)] stdout> results


2024-06-10 00:56:58 +0200   26317 execution.flow     INFO     [relevance_aggregate_variants_results in line None (index starts from 0)] stdout>  


2024-06-10 00:56:58 +0200   26317 execution.flow     INFO     [relevance_aggregate_variants_results in line None (index starts from 0)] stdout> [{'gpt_relevance': 2.0, 'gpt_relevance_pass_rate': 0}]


2024-06-10 00:56:58 +0200   26317 execution.flow     INFO     Node relevance_aggregate_variants_results completes.


In [5]:
test_multi_hf

{'gpt_coherence': 2.0,
 'gpt_fluency': 2.0,
 'gpt_groundedness': 1.0,
 'gpt_relevance': 2.0}

In [6]:

# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name_hf = timestamp+"chat_base_run"
print(run_name_hf)

# create base run in Azure Ai Studio
local_base_run_hf = pf_client.run(
    flow=flow,
    data=data,
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name_hf,
    name=run_name_hf
)
print(local_base_run_hf)

2024_06_10_005658chat_base_run


[2024-06-10 00:56:58 +0200][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run 2024_06_10_005658chat_base_run, log path: /Users/yonghuizhu/.promptflow/.runs/2024_06_10_005658chat_base_run/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=2024_06_10_005658chat_base_run


You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x7ac72b59417c9bf2cfe5e33a2fc4cd9c
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x80c22ea6469857aaee67fcc907d08a94
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xcfb6e53a3b992579dda51a5a4103e32f
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x437abbc7beeaec28996c7ba95fca6050
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xab049d57f5e4b7eb4645f645fbdcd468


name: 2024_06_10_005658chat_base_run
created_on: '2024-06-10T00:56:58.650188'
status: Completed
display_name: 2024_06_10_005658chat_base_run
description:
tags:
properties:
  flow_path: /Users/yonghuizhu/imperial/contoso-chat-backend/contoso-chat_hf
  output_path: /Users/yonghuizhu/.promptflow/.runs/2024_06_10_005658chat_base_run
  column_mapping:
    customerId: ${data.customerId}
    question: ${data.question}
  system_metrics:
    total_tokens: 7469
    prompt_tokens: 6969
    completion_tokens: 500
    duration: 5.373
flow_name: contoso-chat_hf
data: /Users/yonghuizhu/imperial/contoso-chat-backend/data/salestestdata.jsonl
output: /Users/yonghuizhu/.promptflow/.runs/2024_06_10_005658chat_base_run/flow_outputs



In [7]:
pf_client.stream(local_base_run_hf)
details = pf_client.get_details(local_base_run_hf)
details.head(10)

2024-06-10 00:56:58 +0200   26317 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-06-10 00:56:59 +0200   26317 execution.bulk     INFO     Current system's available memory is 1477.828125MB, memory consumption of current process is 269.265625MB, estimated available worker count is 1477.828125/269.265625 = 5
2024-06-10 00:56:59 +0200   26317 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 5, 'estimated_worker_count_based_on_memory_usage': 5}.
2024-06-10 00:56:59 +0200   26317 execution.bulk     INFO     Process name(SpawnProcess-5)-Process id(26339)-Line number(1) start execution.
2024-06-10 00:56:59 +0200   26317 execution.bulk     INFO     Process name(SpawnProcess-2)-Process id(26335)-Line number(0) start execution.
2024-06-10 00:56:59 +0200   26317 execution.bulk     INFO     Process name(SpawnProcess-3)-Process id(26337)-L

,inputs.customerId,inputs.question,inputs.line_number,inputs.chat_history,outputs.answer,outputs.context
0,4,tell me about your hiking jackets,0,[],Hi there 👋 SaraLee 🏞️!\n\nI'd love 💕 sharing o...,"[{'id': '17', 'title': 'RainGuard Hiking Jacke..."
1,1,Do you have any climbing gear?,1,[],"Hey there 👋 JOHN SMITH 🎉,\n\nI see we've got q...","[{'id': '9', 'title': 'SummitClimber Backpack'..."
2,3,Can you tell me about your selection of tents?,2,[],Hey there 👋 Mike 🏕️!\n\nI see we've got our tr...,"[{'id': '15', 'title': 'SkyView 2-Person Tent'..."
3,6,Do you have any hiking boots?,3,[],"Hi there 👋️‍♀🏽**, *Emil*😊y Rodríguez!* \n\nYes...","[{'id': '4', 'title': 'TrekReady Hiking Boots'..."
4,2,What gear do you recommend for hiking?,4,[],"Hi there 👋, Gold member 🏅 *ahem* I mean 😊 Hi a...","[{'id': '10', 'title': 'TrailBlaze Hiking Pant..."


In [8]:

run_name = timestamp+"chat_eval_run"
print(run_name)

run_name_hf = timestamp+"chat_eval_run"
print(run_name_hf)

local_eval_run_variant = pf_client.run(
    flow=eval_flow,
    data=data,  # path to the data file
    run=local_base_run_hf,  # use run as the variant
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    },
    runtime=runtime,
    display_name=run_name,
    name=run_name
)

[2024-06-10 00:57:04 +0200][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run 2024_06_10_005658chat_eval_run, log path: /Users/yonghuizhu/.promptflow/.runs/2024_06_10_005658chat_eval_run/logs.txt


2024_06_10_005658chat_eval_run
2024_06_10_005658chat_eval_run
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=2024_06_10_005658chat_eval_run


You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x00f4d8d13731a27b45e8ebd47356099a
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x65efcf941769182ddd2d3f538b825390
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x35910c40356c5dd117027952f6178cfb
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x4f0c6142cdca28820033cb4903457a33
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xccc9f5bff1ca00dfbc1363ff534ee214


In [9]:

metrics_hf = pf_client.get_metrics(local_eval_run_variant)
import json
print(json.dumps(metrics_hf, indent=4))
pf_client.stream(local_eval_run_variant)

{
    "gpt_coherence": 2.0,
    "gpt_coherence_pass_rate(%)": 0.0,
    "gpt_fluency": 1.8,
    "gpt_fluency_pass_rate(%)": 0.0,
    "gpt_groundedness": 2.6,
    "gpt_groundedness_pass_rate(%)": 40.0,
    "gpt_relevance": 1.8,
    "gpt_relevance_pass_rate(%)": 0.0
}


2024-06-10 00:57:04 +0200   26317 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-06-10 00:57:04 +0200   26317 execution.bulk     INFO     Current system's available memory is 1571.390625MB, memory consumption of current process is 150.65625MB, estimated available worker count is 1571.390625/150.65625 = 10
2024-06-10 00:57:04 +0200   26317 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 5, 'estimated_worker_count_based_on_memory_usage': 10}.
2024-06-10 00:57:04 +0200   26317 execution.bulk     INFO     Process name(SpawnProcess-9)-Process id(26364)-Line number(0) start execution.
2024-06-10 00:57:04 +0200   26317 execution.bulk     INFO     Process name(SpawnProcess-10)-Process id(26365)-Line number(1) start execution.
2024-06-10 00:57:04 +0200   26317 execution.bulk     INFO     Process name(SpawnProcess-8)-Process id(26363)-

In [10]:
pf_client.visualize([local_base_run_hf, local_eval_run_variant])

The HTML file is generated at '/private/var/folders/hf/3mqmr8vd3cs309p9_zg2wcsr0000gn/T/pf-visualize-detail-ykri7qqz.html'.
Trying to view the result in a web browser...


Successfully visualized from the web browser.
